In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn import model_selection
from sklearn import preprocessing



## Load data

In [2]:
data_dir = 'Musk1.xlsx'
test_index_dir = "Musk1.csv_rep1_fold1.txt"


In [3]:
data = pd.read_excel(data_dir)
test_indices  = np.loadtxt(test_index_dir, dtype=int)
train_data = data[~data.bagID.isin(test_indices)]
test_data = data[data.bagID.isin(test_indices)]

In [4]:
x_train = train_data[train_data.columns[3:]].values
normalizer = preprocessing.StandardScaler().fit(x_train)
train_data[train_data.columns[3:]] = normalizer.transform(train_data[train_data.columns[3:]])
test_data[test_data.columns[3:]] = normalizer.transform(test_data[test_data.columns[3:]])

/Users/nimamanaf/opt/anaconda3/envs/q/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/Users/nimamanaf/opt/anaconda3/envs/q/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [5]:
class MilDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.unique_bag_ids = data.bagID.unique()

    def __len__(self):
        return self.data.bagID.nunique()
    
    def __getitem__(self, index):
        bag_id = self.unique_bag_ids[index]
        index_data = self.data[self.data.bagID==bag_id]
        index_features =  torch.tensor(index_data[index_data.columns[3:]].values)
        index_label = torch.tensor(index_data.response.unique())

        return index_features, index_label


In [6]:
train_data = MilDataset(train_data)
test_data = MilDataset(test_data)


In [7]:

train_dl = DataLoader(dataset=train_data, batch_size=1) # Using custom collate_fn mil.collate
test_dl = DataLoader(dataset=test_data, batch_size=1)

## Define model, criterion and optimizer

In [8]:
class mi_Net(nn.Module):
    def __init__(self, input_dim=166, pooling_method="mean"):
        super().__init__()

        self.fc1 = nn.Linear(in_features=input_dim, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=64)
        self.dropout = nn.Dropout(p=0.5, inplace=False)
        self.fc4 = nn.Linear(in_features=64, out_features=1)
        if pooling_method == "mean":
            self.pooling_method = torch.mean
        elif pooling_method == "max":
            self.pooling_method = torch.max
        elif self.pooling_method == "attention":
            raise NotImplementedError
        else:
            raise NotImplementedError
    
    def forward(self, input):    
        
        x = input.float()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.dropout(x) 
        x = torch.sigmoid(self.fc4(x))       
        return self.pooling_method(x, dim=1, keepdim=False)
        

In [10]:
class MI_Net(nn.Module):
    def __init__(self, input_dim=166, pooling_method=""):
        super().__init__()

        self.fc1 = nn.Linear(in_features=input_dim, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=64)
        self.dropout = nn.Dropout(p=0.5, inplace=False)
        self.fc4 = nn.Linear(in_features=64, out_features=1)
    
    def forward(self, input):    
        
        x = input.float()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.dropout(x) 
        x = torch.mean(x, dim=1, keepdim=False)
        x = self.fc4(x)

        return torch.sigmoid(x)



In [33]:
class MI_Net_RC(nn.Module):
    def __init__(self, input_dim=166, dropout=False, pooling_method="mean"):
        super().__init__()

        self.fc1 = nn.Linear(in_features=input_dim, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=128)
        self.fc4 = nn.Linear(in_features=128, out_features=1)
        self.dropout = dropout
        if dropout:
            self.dropout_1 = nn.Dropout(p=0.5, inplace=False)
            self.dropout_2 = nn.Dropout(p=0.5, inplace=False)
            self.dropout_3 = nn.Dropout(p=0.5, inplace=False)
        
        if pooling_method == "mean":
            self.pooling_method = torch.mean
        elif pooling_method == "max":
            self.pooling_method = torch.max
        elif self.pooling_method == "attention":
            raise NotImplementedError
        else:
            raise NotImplementedError
        
    def forward(self, input):    
        
        x = input.float()
        x = F.relu(self.fc1(x))
        if self.dropout:
            x = self.dropout_1(x)
        r1 = self.pooling_method(x, dim=1, keepdim=False)
        x = F.relu(self.fc2(x))
        if self.dropout:
            x = self.dropout_2(x)
        r2 = self.pooling_method(x, dim=1, keepdim=False)
        x = F.relu(self.fc3(x))
        if self.dropout:
            x = self.dropout_3(x)
        r3 = self.pooling_method(x, dim=1, keepdim=False)        
        x = r1 + r2 + r3
        x = self.fc4(x)

        return torch.sigmoid(x)

In [34]:
model = MI_Net()
model = mi_Net()
model = MI_Net_RC()

## Hyper-Parameters 

In [35]:

batch_size = 1
# Optimizer parameters from https://github.com/yanyongluan/MINNs/blob/master/MI_Net.py
learning_rate = 5e-4
weight_decay = 1e-4
momentum = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)
criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()


## Train model

In [36]:
import time

# Training parameters
epochs = 50

start = time.time()
print('TRAINING:')

# Tensor for collecting losses over batches
train_losses = []

for epoch in range(epochs): 
    for features, labels in train_dl:
        labels[labels==-1] = 0  # replace -1 classes with Zero
        pred = model(features)
        loss = criterion(pred, labels.float())
        
        # Update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Save loss on this batch
        train_losses.append(loss.float())
    
    # Compute avarega loss on this epoch
    train_loss = torch.mean(torch.tensor(train_losses), dim = 0, keepdim = True)
    
    # Clear tensor for saving losses over batches
    train_losses = []

    # Print info about learning every 100 epochs
    if (epoch+1)%100 == 0:
        print('[{}/{}] | train_loss: {}'.format(epoch+1, epochs, train_loss.item()))

print('Finished training - elapsed time: {}'.format(time.time() - start))

TRAINING:
Finished training - elapsed time: 5.4081878662109375


## Evaluation

In [37]:
model.eval()
train_acc = []
for features, labels in train_dl:
        labels[labels==-1] = 0  # replace -1 classes with Zero
        pred = model(features)
        train_acc.append(pred.round().detach().item() ==labels.item())

print(f"the pred acc is {sum(train_acc)/len(train_acc)}")
    

the pred acc is 1.0


In [38]:
model.eval()
test_acc = []
for features, labels in test_dl:
    labels[labels==-1] = 0  # replace -1 classes with Zero
    pred = model(features) # get probability of the bag
    
    test_acc.append(pred.round().detach().item() ==labels.item())

print(f"the pred acc is {sum(test_acc)/len(test_acc)}")


the pred acc is 0.9


In [17]:
from sklearn import metrics

def eer(pred, labels):
    fpr, tpr, threshold = metrics.roc_curve(labels.detach(), pred.detach(), pos_label=1)
    fnr = 1 - tpr
    EER_fpr = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    EER_fnr = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
    return EER_fpr, EER_fnr

def accuracy(pred, target, threshold = 0):
    pred = pred.detach().np()
    target = target.detach().np()

    pred[pred >= threshold] = 1
    pred[pred < threshold] = -1

    return np.sum(target == pred)/target.shape[0]


tensor([-1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1, -1, -1,  1, -1,  1, -1, -1,
         1,  1, -1, -1,  1, -1,  1, -1,  1,  1, -1,  1, -1,  1,  1,  1, -1, -1,
        -1,  1, -1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1,
         1,  1, -1,  1,  1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1,  1,  1,  1,
         1])